In [ ]:
import pandas as pd
import glob
from azure.storage.blob.blockblobservice import BlockBlobService
import matplotlib.pyplot as plt
import numpy as np

pd.__version__

## Data download

In [ ]:
block_blob_service = BlockBlobService(account_name='activityprofiles', account_key='vx5JFWqh2ajuF6iQey3xj0o3BJ0o2EZaPxtS1ME28a/9y1+e06SYXnYWoMqhh4OaY5ZRf4wKmZXgJf3im0Bmuw==')

generator = block_blob_service.list_blobs('app')

fp = open('test.csv', 'bw')

for blob in generator:
    # Using `get_blob_to_bytes`
    b = block_blob_service.get_blob_to_bytes('app', blob.name)
    fp.write(b.content)
    # Or using `get_blob_to_stream`
    # service.get_blob_to_stream(container_name, blob.name, fp)

fp.flush()
fp.close()

## Data cleaning

In [ ]:
lengths = []
file = open("test.csv")
lines = file.readlines()
file.close()

for line in lines:
    elements = line.split(",")
    lengths.append(len(elements))

pd.Series(lengths).value_counts()

In [ ]:
# convert 32 to 18 length
def bad_line_handler(bad_line):
    elements = bad_line[:3]
    for i in range(0, len(bad_line[3:]), 2):
        elements.append(f"{bad_line[i+3]}.{bad_line[i+4]}")
    return elements

# test
correct_line = bad_line_handler("22.03.2022 20:19:37.872,Flavio,Bicycling,0,3985294,-0,9182853,0,3608138,5,368,45,445,7,2712,0,1343864,2,928402,-0,6817055,-0,5596727,-0,4046215,0,2832775,0,6654648,47,47692593,8,21402806".split(","))
print(len(correct_line))

lines_new = []

# convert 32 elements to 18, drop 31
for i, line in enumerate(lines):
    elements = lines[i].split(",")
    if(len(elements) == 18):
        lines_new.append(line)
    elif(len(elements) == 33):
        correct_line = bad_line_handler(elements)
        lines_new.append(str.join(",", correct_line))
    elif(len(elements) == 32):
        pass
    else:
        raise (f"ERROR: unknown error at line {i}")

print(len(lines_new))

In [ ]:
file = open("test.csv", "w")
file.writelines(lines_new)
file.close()

del lines
del lines_new

In [ ]:
df = pd.read_csv('test.csv', index_col=False, header=None) #engine="python", on_bad_lines=bad_line_handler
df.columns = ['time','name','activity','acc_x','acc_y','acc_z','mag_x','mag_y','mag_z','gyr_x','gyr_y','gyr_z','ori_x','ori_y','ori_z','ori_w','lat','long']
print(df.shape)
df

In [ ]:
 # drop test entry
 df_clean = df.loc[df['time'] != 'Test04.03.2022 07:48:54.917', :]

In [ ]:
# convert datetime format, yess this takes 5 min. :/
df_clean.loc[:, 'time'] = pd.to_datetime(df_clean.loc[:, 'time'] + "000") # format="%d.%m.%Y %H:%M.%S.%f"

In [ ]:
# drop all data before 3.3.2022, because of incorrect data loading
df_clean = df_clean[df_clean['time'] > pd.to_datetime('03.03.2022 08:00:00')]

In [ ]:
df_clean['name'].unique()

In [ ]:
# drop invalid user
df_clean = df_clean[(df_clean['name'].isin(['Raphi', 'pascal', 'Flavio', 'Ronny', 'Simon']))]

In [ ]:
df_clean.to_csv("df_clean.csv")

In [ ]:
del df

## Data vlisualization

In [ ]:
df_clean = pd.read_csv("df_clean.csv")
df_clean.shape

In [ ]:
df_clean['activity'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
df_clean['activity'].value_counts()

In [ ]:
n = np.arange(len(df_clean['activity'].unique()))

fig, ax = plt.subplots(figsize=(22, 5))
names = []
for i, name in enumerate(df_clean['name'].unique()):
    ax.bar(n + (i/len(df_clean['name'].unique())/2 - 0.25), df_clean.loc[df_clean['name'] == name, 'activity'].value_counts().reindex(df_clean['activity'].unique(), fill_value=0), width=0.1)
    names.append(name)

# minline: 2h/5=24min, 1440s per person, 20meas/sec -> 28'800 measurements
plt.xticks(n, list(df_clean['activity'].unique()), rotation=45)
plt.legend(names)
ax.hlines(y=28800, xmin=-0.5, xmax=6.5, colors='green')
#plt.legend(a[1].unique())
plt.plot()

In [ ]:
# coordinates visualization
fig, ax = plt.subplots(figsize=(15, 5))
ax.scatter(df_clean['lat'].values, df_clean['long'].values)
#ax.hist(a[17].value_counts(), bins=50)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(22, 5))
ax.plot(df_clean.loc[(df_clean['name'] == 'pascal') & (df_clean['activity'] == 'Stairway'), ['mag_x', 'mag_y', 'mag_z']])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(22, 5))
ax.plot(df_clean.loc[(df_clean['name'] == 'pascal') & (df_clean['activity'] == 'Elevatoring'), ['mag_x', 'mag_y', 'mag_z']])
plt.show()